In [1]:
import pandas as pd
import numpy as np

import warnings
import io,base64,os,json,re
import xml.etree.ElementTree as ET


import xml.etree.ElementTree as ET
import pandas as pd
from collections import defaultdict
import os
import import_ipynb 


In [2]:
import cx_Oracle as cx

import tkinter as tk
from tkinter import filedialog, messagebox, ttk

import webbrowser

# initalise the tkinter GUI
root = tk.Tk()



root.lift()
root.attributes('-topmost',True)
root.after_idle(root.attributes,'-topmost',False)
root.geometry("400x300") # set the root dimensions
root.pack_propagate(False) # tells the root to not let the widgets inside it determine its size.
root.resizable(0, 0) # makes the root window fixed in size.
root.configure(bg='sky blue')




# Frame for open file dialog
file_frame = tk.LabelFrame(root, text="SCF compare")
file_frame.place(height=200, width=400, relx=0)
file_frame.configure(bg='sky blue')

# Buttons
upload_file=tk.Entry(file_frame,width=36)
upload_file.place(rely=0.20, relx=0.35)





Button1=tk.Button(file_frame, text="Browse A scf-1", command=lambda: File_dialog_src())
Button1.place(rely=0.18, relx=0.10)

upload_file_trg=tk.Entry(file_frame,width=36)

upload_file_trg.place(rely=0.40, relx=0.35)

But1=tk.Button(file_frame, text="Browse A scf-2", command=lambda: File_dialog_trg())
But1.place(rely=0.35, relx=0.10)







button2 = tk.Button(file_frame, text="Load_4G", command=lambda:Load_data())
button2.place(rely=0.8, relx=0.3)


button3 = tk.Button(file_frame, text="Load_5G",command=lambda:load_data_5G())
button3.place(rely=0.8, relx=0.6)


def File_dialog_src():
    
    global filename_src
    """This Function will open the file explorer and assign the chosen file path to label_file"""

    filename_src = filedialog.askopenfilename(initialdir="/",
                                          title="Select A File",
                                          filetype=(("XML files", "*.xml"),("All Files", "*.*")))
    upload_file.insert(tk.END, filename_src)
    return None
def File_dialog_trg():
    
    global filename_trg
    """This Function will open the file explorer and assign the chosen file path to label_file"""

    filename_trg = filedialog.askopenfilename(initialdir="/",
                                          title="Select A File",
                                          filetype=(("XML files", "*.xml"),("All Files", "*.*")))
    upload_file_trg.insert(tk.END, filename_trg)
                           
    return None

def load_data_5G():      
    #upload xmls files
    
    def _tree_xml(path):
        tree=ET.parse(path)
        return tree.getroot()

    root_src=_tree_xml(filename_src)
    root_trg=_tree_xml(filename_trg)
    
    #list class reference
    class_ref=['NRBTS','NRDRB_RLC_AM','NRSCTP','TRACKINGAREA','BWP_PROFILE','PDCCH','NRDRB_PDCP','NRDRB','NRDRB_5QI','PDCCH_CONFIG_COMMON','PDSCH','PDCCH_CONFIG_DEDICATED','NRDRB_QCI','NRANR','NRANRPR','NRSYSINFO_PROFILE_NSA']


    class_ref=["com.nokia.srbts.nrbts:"+w for w in class_ref]
    class_ref.extend([ 'com.nokia.srbts.tnl:TNLSVC',
     'com.nokia.srbts.tnl:PMTNLINT','com.nokia.srbts.tnl:TNL','com.nokia.srbts.tnl:ETHSVC','com.nokia.srbts.tnl:ETHIF',\
     'com.nokia.srbts.tnl:VLANIF','com.nokia.srbts.tnl:ETHLK','com.nokia.srbts.tnl:L2SWI','com.nokia.srbts.tnl:BRGPRT',\
     'com.nokia.srbts.tnl:IBRGPRT','com.nokia.srbts.tnl:IPNO','com.nokia.srbts.tnl:IPIF','com.nokia.srbts.tnl:IPADDRESSV4',\
     'com.nokia.srbts.tnl:IPRT','com.nokia.srbts.tnl:IPRTV6','com.nokia.srbts.tnl:QOS','com.nokia.srbts.tnl:DSCP2PCPMAP',\
     'com.nokia.srbts.tnl:DSCP2QMAP','com.nokia.srbts.tnl:FSTSCH','com.nokia.srbts.tnl:ETHAPP','com.nokia.srbts.tnl:OAMMD'])
    #classes xml_file
    src_class=[elm.attrib['class'] for elm in root_src.findall(".//{raml21.xsd}managedObject")]
    trg_class=[elm.attrib['class'] for elm in root_trg.findall(".//{raml21.xsd}managedObject")]

    #intersection file
    liste_class=[x for x in class_ref if ((x in src_class) and (x in trg_class))]
    missing_class=[x for x in class_ref if x not in list(src_class)]


    #list class-cell-level

    class_lncel_ref=['NRCELL','NRCELL_FDD','NRDSSLTE','NRCELLGRP','NRDRB',
    'NRDRB_MAC','NRDU','NRULCLPC_PROFILE',
     'NRCUUP'] 
    class_lncel=["com.nokia.srbts.nrbts:"+w for w in class_lncel_ref]
    class_lncel=[x for x in class_lncel if ((x in src_class) and (x in trg_class))]
    missing_class_cell=[x for x in class_lncel_ref if x not in list(src_class)]
    dictio={"missing_class_bts_src":[x for x in class_ref if x not in list(src_class)],'missing_class_bts_trg':[x for x in class_ref if x not in list(trg_class)],
       'missing_class_cell_src':[x for x in class_lncel_ref if x not in list(src_class)],
       'missing_class_cell_trg':[x for x in class_lncel_ref if x not in list(trg_class)]
      }

    #check versions:
    for i,j in zip(root_src.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']"),root_trg.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']")):
            if i.attrib["version"]== j.attrib["version"]:
                msg="you're comparing same versions"
            else:
                msg="you're comparing differents versions!"

    #LBTS_name
    for i,j in zip(root_src.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']/{raml21.xsd}p/[@name='btsName']"),root_trg.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']/{raml21.xsd}p/[@name='btsName']")):
        src_bts=i.text
        trg_bts=j.text
        if src_bts==trg_bts:
            trg_bts+="_v2"
    print(src_bts)
#Ajout d’un prefixe pour les paramètres d’un structure.
    
    def file_xml(root,classe):
        for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list"):
                print(i.attrib)
                for el in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list/[@name='"+j.attrib["name"]+"']/{raml21.xsd}item/{raml21.xsd}p"):
                    el.attrib["name"]=j.attrib['name']+"-"+el.attrib["name"]
                    print(el.attrib["name"])
# create dict de paramètres et le tranformer en DF

        dicte=defaultdict(list)
        df_lnbts=pd.DataFrame()
        for elm in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            dicte['distName'].append(elm.attrib['distName'])
            dicte['class'].append(elm.attrib['class'])
        for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/{raml21.xsd}p"):
            dicte[i.attrib["name"]].append(i.text)
        for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/{raml21.xsd}list/{raml21.xsd}item/{raml21.xsd}p"):
            dicte[j.attrib["name"]].append(j.text)
        return pd.DataFrame.from_dict(dicte,orient='index').transpose().fillna(method='ffill')


    #dictionnary of dfs for source
    src={}
    for i in liste_class:
        src["df_"+i]=file_xml(root_src,i)

    #dictionnary of dfs for target 
    trg={}
    for i in liste_class:
        trg["df_"+i]=file_xml(root_trg,i)
     #parsing files
    def file_xml_lncel(root,classe):
        for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list"):
                for el in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list/[@name='"+j.attrib["name"]+"']/{raml21.xsd}item/{raml21.xsd}p"):
                    el.attrib["name"]=j.attrib['name']+"-"+el.attrib["name"]


        df=pd.DataFrame()
        for elm in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):

            dicte=defaultdict(list)
            dicte['distName'].append(elm.attrib['distName'])
            dicte['class'].append(elm.attrib['class'])
            for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+elm.attrib['distName']+"']/{raml21.xsd}p"):
                dicte[i.attrib["name"]].append(i.text)
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+elm.attrib['distName']+"']/{raml21.xsd}list/{raml21.xsd}item/{raml21.xsd}p"):
                dicte[j.attrib["name"]].append(j.text)
            df=df.append(pd.DataFrame.from_dict(dicte,orient='index').transpose(),ignore_index=True)

        df=df[df["distName"].notnull()]
        class_ho={"NOKLTE:CAREL":"lcrId","NOKLTE:IRFIM":"dlCarFrqEut","NOKLTE:LNHOW":"utraCarrierFreq","NOKLTE:REDRT":"redirFreqUtra"}

        df['bande']=df["distName"].map(lambda x: x.split('/NRCELL-')[-1])

        if classe in list(class_ho.keys()):
            df['bande']=df['bande'].str[:2]+">"+df[class_ho[classe]]
        df = df.set_index('bande')[df.set_index('bande').index.notnull()]
        return df

    
    #dictionnary of dfs for sources&targets
    src_cell={}
    trg_cell={}
    for i in class_lncel:
        src_cell["df_"+i]=file_xml_lncel(root_src,i)
        trg_cell["df_"+i]=file_xml_lncel(root_trg,i)
        print(src_cell["df_"+i]["distName"])
        
    def parsing_xml(file_bts,file_cell):
        file={**file_bts,**file_cell}
        s=[]
        liste=[]
        pars_src=pd.DataFrame()
        for j in file.keys():
            for i,row in file[j].iterrows():
                df=row.transpose()
                s.append(df)
                pars_bts=pd.concat(s).reset_index()

                pars_bts["bande"]=i
                pars_bts["class"]=j.split(":")[-1]
            liste.append(pars_bts)
        return(pd.concat(liste).reset_index())

    src_parse=parsing_xml(src,src_cell)
    trg_parse=parsing_xml(trg,trg_cell)

    # rename columns
    try:
        col=["Parameters",trg_bts,src_bts]
        for i, j in zip(src_parse.columns[1:3],["Parameters",src_bts]):
            src_parse.rename(columns={i:j},inplace=True)
        for i, j in zip(trg_parse.columns[1:3],["Parameters",trg_bts]):
            trg_parse.rename(columns={i:j},inplace=True)
    except:
        print("error, syntax in columns, or empty files")
        
        
    def _compare_(df_src,df_trg):

        df_trg.drop(df_trg[df_trg.index.isin(list(set(df_src.index).symmetric_difference(df_trg.index)))].index,inplace=True) 
        df_src.drop(df_src[df_src.index.isin(list(set(df_trg.index).symmetric_difference(df_src.index)))].index,inplace=True) 
#aligner les paramètres avec le même index sur chaque scf
        for i in list(df_trg.columns.difference(df_src.columns)):
            if df_trg.columns.get_loc(i)>len(df_src.columns):
                df_src[i]=None
            else :
                df_src.insert(df_trg.columns.get_loc(i),i,None)

        print(df_src)
        for i in list(df_src.columns.difference(df_trg.columns)):
            print(df_src.columns.get_loc(i))
            if df_src.columns.get_loc(i)>len(df_trg.columns):
                df_trg[i]=None
            else :
                df_trg.insert(df_src.columns.get_loc(i),i,None)

        df_src=df_src[df_src["distName"].notnull()]
        df_trg=df_trg[df_trg["distName"].notnull()]

        df_trg=df_trg[df_src.columns]
        return df_src.compare(df_trg)

#creation d'un dictionnaire de delta pour chaque class
    check={}
    for i in liste_class:
        try:
            check["check_"+i]=_compare_(src['df_'+i],trg['df_'+i])

        except ValueError as e:
            print("Object-"+i+"- doesnt have the same size in src and trg")
            pass
    print(check["check_"+i].head())
    #custolmizing result selon la forme de output demandée
    liste_bts=[]
    for j in check.keys():
        for i in check[j].index:
            try:
                print(i)
                df=check[j][check[j].index==i]
                df_y=df.transpose().reset_index().pivot(index="level_0", columns="level_1", values=i)
                df_y["bande"]=i
                df_y["class"]=j.split(":")[1]
                liste_bts.append(df_y)
            except:
                print("Small issue in displaying result for object:"+j)
                pass
    try:        
        df_bts=pd.concat(liste_bts).reset_index()
        #rename columns
        col=["Parameters",trg_bts,src_bts]
        for i, j in zip(df_bts.columns[:-1],col):
            df_bts.rename(columns={i:j},inplace=True)
            print(i,j)
        df_bts = df_bts.drop_duplicates(subset=['Parameters',trg_bts,src_bts,'class'], keep="first")
    except:
        print("result of comparaison is empty or you're comparing the same files")

    print( df_bts.head())
   

    def compare_lncel(df_src,df_trg):
        print(list(df_src.index))
        df_src
        df_trg.drop(df_trg[df_trg.index.isin(list(set(df_src.index).symmetric_difference(df_trg.index)))].index,inplace=True) 
        df_src.drop(df_src[df_src.index.isin(list(set(df_trg.index).symmetric_difference(df_src.index)))].index,inplace=True) 


        for i in list(df_trg.columns.difference(df_src.columns)):
            if df_trg.columns.get_loc(i)>len(df_src.columns):
                df_src[i]=None
            else :
                df_src.insert(df_trg.columns.get_loc(i),i,None)

        for i in list(df_src.columns.difference(df_trg.columns)):
            if df_src.columns.get_loc(i)>len(df_trg.columns):
                df_trg[i]=None
            else :
                df_trg.insert(df_src.columns.get_loc(i),i,None)  
        print(df_trg["class"])    
        print(len(df_trg.columns))
        df_trg=df_trg[df_src.columns]
        #df_trg=df_trg.reindex(df_src.index)
        print(df_trg.shape)
        print(df_src.shape)
        print(list(df_src.index))
        print(list(df_trg.index))
        return df_src.compare(df_trg)



    # creatin comparing dictionnary
    check_lncel={}
    for i in class_lncel:
        try:
            check_lncel["check_"+i]=compare_lncel(src_cell['df_'+i],trg_cell['df_'+i])
        except ValueError as e:
            print("Object-"+i+"- doesnt have the same size in src and trg")
            pass

    #custolmizing result
    liste_parent=[]
    for j in check_lncel.keys():
        for i,row in check_lncel[j].iterrows():
            print(j)
            print(i)
            try :
                df=row
                df_x=df.transpose().reset_index().pivot(index="level_0", columns="level_1", values=i)
                df_x["bande"]=i
                df_x["class"]=j.split(":")[1]
                liste_parent.append(df_x)
            except:
                print("issue when diplaying comparaison for object"+j)
    try:
        df_cell=pd.concat(liste_parent).reset_index()

        #rename columns-cell
        col=["Parameters",trg_bts,src_bts]
        for i, j in zip(df_cell.columns[:3],col):
            df_cell.rename(columns={i:j},inplace=True)
            print(i,j)
        df_bts=df_bts[df_bts["Parameters"]!="distName"]
        df_cell=df_cell[df_cell["Parameters"]!="distName"]
    except:
        print("nothing to compare")

   


    #savin in excel file
    with pd.ExcelWriter('D:\scf_compare_RT_v8'+src_bts+'_'+trg_bts+'.xlsx') as writer:  
        df_bts[["Parameters","class",src_bts,trg_bts]].to_excel(writer, sheet_name='BTS-level')
        df_cell[(df_cell[src_bts].notnull()) | (df_cell[trg_bts].notnull())][["class","bande","Parameters",src_bts,trg_bts]].to_excel(writer, sheet_name="Cell-level")
        pd.concat([df_bts[["class","bande","Parameters",src_bts,trg_bts]],df_cell[(df_cell[src_bts].notnull()) | (df_cell[trg_bts].notnull())][["class","bande","Parameters",src_bts,trg_bts]]]).to_excel(writer,sheet_name="all")
        pd.DataFrame.from_dict(dictio,orient='index').transpose().to_excel(writer, sheet_name="Missing_object")

        #src_parse[src_parse.columns[1:]].to_excel(writer, sheet_name=src_bts+'-Parsing',index=False)
        #trg_parse[trg_parse.columns[1:]].to_excel(writer, sheet_name=trg_bts+'-Parsing',index=False)
    root.destroy() 
   
    return None

def Load_data():
    def _tree_xml(path):
        tree=ET.parse(path)
        return(tree.getroot())
    root_src=_tree_xml(filename_src)
    root_trg=_tree_xml(filename_trg)

 
    #list class reference
    class_ref=['LNBTS','ANR','ANRPRL','ANRPRW','CADPR','CAGENB','CTRLTS','DPCLPC','LBPUCCHRDPR','LNBTS_FDD','MODPR','NRDCDPR','NRDCPR','NRMEASDPR','NRMEASPR','PMRNL','RIM','SCTP','TRSNW','ULCOMP']
    class_ref=["NOKLTE:"+w for w in class_ref]
    #classes xml_file
    src_class=[elm.attrib['class'] for elm in root_src.findall(".//{raml21.xsd}managedObject")]
    trg_class=[elm.attrib['class'] for elm in root_trg.findall(".//{raml21.xsd}managedObject")]
    #intersection file
    liste_class=[x for x in class_ref if ((x in src_class) and (x in trg_class))]
    missing_class=[x for x in class_ref if x not in list(src_class)]
    missing_class
    #list class-cell-level
    class_lncel_ref=['NOKLTE:LNCEL','NOKLTE:LNCEL_FDD','NOKLTE:DRX','NOKLTE:CAREL','NOKLTE:SDRX','NOKLTE:SIB','NOKLTE:IRFIM','NOKLTE:REDRT','NOKLTE:LNHOW'] 
    class_lncel=[x for x in class_lncel_ref if ((x in src_class) and (x in trg_class))]
    missing_class_cell=[x for x in class_lncel_ref if x not in list(src_class)]


    dictio={"missing_class_bts_src":[x for x in class_ref if x not in list(src_class)],'missing_class_bts_trg':[x for x in class_ref if x not in list(trg_class)],
       'missing_class_cell_src':[x for x in class_lncel_ref if x not in list(src_class)],
       'missing_class_cell_trg':[x for x in class_lncel_ref if x not in list(trg_class)]
      }


     #check versions:
    for i,j in zip(root_src.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']"),root_trg.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']")):
        if i.attrib["version"]== j.attrib["version"]:
            msg="you're comparing same versions"
        else:
            msg="you're comparing differents versions!"
            
    #LBTS_name
    for i,j in zip(root_src.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']/{raml21.xsd}p/[@name='btsName']"),root_trg.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']/{raml21.xsd}p/[@name='btsName']")):
        src_bts=i.text
        trg_bts=j.text
        if src_bts==trg_bts:
            trg_bts+="_V2"
             #print(src_bts)
    def file_xml(root,classe):
        for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list"):
                #print(i.attrib)
                for el in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list/[@name='"+j.attrib["name"]+"']/{raml21.xsd}item/{raml21.xsd}p"):
                    el.attrib["name"]=j.attrib['name']+"-"+el.attrib["name"]
                    #print(el.attrib["name"])    
        dicte=defaultdict(list)
   
        df_lnbts=pd.DataFrame()
        for elm in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            dicte['distName'].append(elm.attrib['distName'])
            dicte['class'].append(elm.attrib['class'])
            for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/{raml21.xsd}p"):
                dicte[i.attrib["name"]].append(i.text)
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/{raml21.xsd}list/{raml21.xsd}item/{raml21.xsd}p"):
                dicte[j.attrib["name"]].append(j.text)
            
        df=pd.DataFrame.from_dict(dicte,orient='index').transpose().fillna(method='ffill').reset_index()
        class_bts={"NOKLTE:LNBTS":"reqFreqBands-bandNumber","NOKLTE:NRDCPR":"dynTriggerLteNrDcConfList-lcrId","NOKLTE:NRDCDPR":"dynTriggerLteNrDcConfList-lcrId"} 
        
        if (classe in list(class_bts.keys())):
            if (class_bts[classe] in df.columns):
                #print(df[class_bts[classe]])
                df['bande']=">"+df[class_bts[classe]] 
            else:
                df['bande']=classe+"-"+df['index'].astype('str')
            
        else:
            df['bande']=df["distName"].map(lambda x: x.split('/')[-1])+"-"+df['index'].astype('str')
            
        return df.set_index("bande")

      #dictionnary of dfs for source

    src={}
    for i in liste_class:
        src["df_"+i]=file_xml(root_src,i)
    #print(src["df_NOKLTE:LNBTS"])
    
     #dictionnary of dfs for target 
    trg={}
    for i in liste_class:
        trg["df_"+i]=file_xml(root_trg,i)
        
    
    def file_xml_lncel(root,classe):
        for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list"):
                for el in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list/[@name='"+j.attrib["name"]+"']/{raml21.xsd}item/{raml21.xsd}p"):
                    el.attrib["name"]=j.attrib['name']+"-"+el.attrib["name"]
                
    
        df=pd.DataFrame()
        for elm in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            dicte=defaultdict(list)
            dicte['distName'].append(elm.attrib['distName'])
            dicte['class'].append(elm.attrib['class'])
            for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+elm.attrib['distName']+"']/{raml21.xsd}p"):
                dicte[i.attrib["name"]].append(i.text)
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+elm.attrib['distName']+"']/{raml21.xsd}list/{raml21.xsd}item/{raml21.xsd}p"):
                dicte[j.attrib["name"]].append(j.text)
            df=df.append(pd.DataFrame.from_dict(dicte,orient='index').transpose(),ignore_index=True)
        df=df[df["distName"].notnull()]
        
        class_ho={"NOKLTE:CAREL":"lcrId","NOKLTE:IRFIM":"dlCarFrqEut","NOKLTE:LNHOW":"utraCarrierFreq","NOKLTE:REDRT":"redirFreqUtra"}
    
        df['bande']=df["distName"].map(lambda x: x.split('LNCEL-')[-1])
        #print(df.head())
    
        if classe in list(class_ho.keys()):
            df['bande']=df['bande'].str[:2]+">"+df[class_ho[classe]]
        df = df.set_index('bande')[df.set_index('bande').index.notnull()]
        return df

    class_lncel=['NOKLTE:LNCEL','NOKLTE:LNCEL_FDD','NOKLTE:DRX','NOKLTE:SDRX','NOKLTE:CAREL','NOKLTE:SIB','NOKLTE:IRFIM','NOKLTE:REDRT','NOKLTE:LNHOW'] 

    #dictionnary of dfs for sources&targets
    src_cell={}
    trg_cell={}
    for i in class_lncel:
        src_cell["df_"+i]=file_xml_lncel(root_src,i)
        trg_cell["df_"+i]=file_xml_lncel(root_trg,i)
    #print(src_cell['df_NOKLTE:LNCEL'])
        #print(src_cell["df_"+i]["distName"])
     #comparing lncel level
    def parsing_xml(file_bts,file_cell):
        file={**file_bts,**file_cell}
        print(file.keys())
        s=[]
        liste=[]
        pars_src=pd.DataFrame()
        for j in file.keys():
            for i,row in file[j].iterrows():
                df=row.transpose()
                s.append(df)
                pars_bts=pd.concat(s).reset_index()

                pars_bts["bande"]=i
                pars_bts["class"]=j.split(":")[-1]
                pars_bts.rename(columns={pars_bts.columns[1]:0},inplace=True)
                liste.append(pars_bts)
        return(pd.concat(liste).reset_index())

    src_parse=parsing_xml(src,src_cell)
    trg_parse=parsing_xml(trg,trg_cell)

    # rename columns
    try:
        col=["Parameters",trg_bts,src_bts]
        for i, j in zip(src_parse.columns[1:3],["Parameters",src_bts]):
            src_parse.rename(columns={i:j},inplace=True)
        for i, j in zip(trg_parse.columns[1:3],["Parameters",trg_bts]):
            trg_parse.rename(columns={i:j},inplace=True)
    except:
        print("error, syntax in columns, or empty files")

    
    def _compare_(df_src,df_trg):
            
        class_bts={"NOKLTE:LNBTS":"reqFreqBands-bandNumber","NOKLTE:NRDCPR":"dynTriggerLteNrDcConfList-lcrId","NOKLTE:NRDCDPR":"dynTriggerLteNrDcConfList-lcrId"} 
        classe=df_src["class"].unique()[0]
        df_src=df_src.reset_index()
        df_trg=df_trg.reset_index()
        print(classe)
        if (classe in list(class_bts.keys())):
            print(class_bts[classe])
            if (class_bts[classe] in df_src.columns) and (class_bts[classe] in df_trg.columns):
                 
                #print(df[class_bts[classe]])
                
                df_src['bande']=">"+df_src[class_bts[classe]] 
                df_trg['bande']=">"+df_trg[class_bts[classe]] 
                print(df_trg.head())
            else:
                df_src['bande']=classe+"-"+df_src['index'].astype('str')
                df_trg['bande']=classe+"-"+df_trg['index'].astype('str')
                print(df_trg.head())
            df_src=df_src.set_index('bande')
            df_trg=df_trg.set_index('bande')
        #print(df_src.set_index.head(1))    
        #print(df_trg.set_index.head(1))
        
        df_trg.drop(df_trg[df_trg.index.isin(list(set(df_src.index).symmetric_difference(df_trg.index)))].index,inplace=True) 
        df_src.drop(df_src[df_src.index.isin(list(set(df_trg.index).symmetric_difference(df_src.index)))].index,inplace=True) 

    
        for i in list(df_trg.columns.difference(df_src.columns)):
            if df_trg.columns.get_loc(i)>len(df_src.columns):
                df_src[i]=None
            else :
                df_src.insert(df_trg.columns.get_loc(i),i,None)
             
        #print(df_src)
        for i in list(df_src.columns.difference(df_trg.columns)):
            #print(df_src.columns.get_loc(i))
            if df_src.columns.get_loc(i)>len(df_trg.columns):
                df_trg[i]=None
            else :
                df_trg.insert(df_src.columns.get_loc(i),i,None)
    
        df_trg=df_trg[df_src.columns]
        df_trg=df_trg.reindex(df_src.index)
            
        """df_src=df_src[df_src["distName"].notnull()]
        df_trg=df_trg[df_trg["distName"].notnull()]"""
    
        df_trg=df_trg[df_src.columns]
        return df_src.compare(df_trg)

    check={}
    for i in liste_class:
        try:
            check["check_"+i]=_compare_(src['df_'+i],trg['df_'+i])
        except ValueError as e:
            #print("Object-"+i+"- doesnt have the same size in src and trg")
            pass
    #print(check.keys())

    # delete columns with  just name differences

    keys=list(check.keys())
    for i in keys:
        #print(i)
        if len(check[i].columns)<=2:
            del check[i]
    

     #custolmizing result
    liste_bts=[]
    for j in check.keys():
        for i in check[j].index:
            try:
                df=check[j][check[j].index==i]
                df_y=df.transpose().reset_index().pivot(index="level_0", columns="level_1", values=i)
                df_y["bande"]=i
                df_y["class"]=j.split(":")[1]
                liste_bts.append(df_y)
            except:
                #print("Small issue in displaying result for object:"+j)
                pass
    #print(pd.concat(liste_bts).reset_index())
        
    try:        
        df_bts=pd.concat(liste_bts).reset_index()
        
        #rename columns
        col=["Parameters",trg_bts,src_bts]
        for i, j in zip(df_bts.columns[:-1],col):
            df_bts.rename(columns={i:j},inplace=True)
            #print(df_bts.columns[:-1])
        df_bts = df_bts.drop_duplicates(subset=['Parameters',trg_bts,src_bts,'class'], keep="first")
    except:
        print("result of comparaison is empty or you're comparing the same files")
    #lncell part

    #comparing lncel level

    def compare_lncel(df_src,df_trg):
        #print(list(df_src.index))
        df_trg.drop(df_trg[df_trg.index.isin(list(set(df_src.index).symmetric_difference(df_trg.index)))].index,inplace=True) 
        df_src.drop(df_src[df_src.index.isin(list(set(df_trg.index).symmetric_difference(df_src.index)))].index,inplace=True) 

        for i in list(df_trg.columns.difference(df_src.columns)):
            if df_trg.columns.get_loc(i)>len(df_src.columns):
                df_src[i]=None
            else :
                df_src.insert(df_trg.columns.get_loc(i),i,None)
             
        for i in list(df_src.columns.difference(df_trg.columns)):
            if df_src.columns.get_loc(i)>len(df_trg.columns):
                df_trg[i]=None
            else :
                df_trg.insert(df_src.columns.get_loc(i),i,None)  
        #print(df_trg["class"])    
        #print(len(df_trg.columns))
        df_trg=df_trg[df_src.columns]
        df_trg=df_trg.reindex(df_src.index)
        #print(df_trg.shape)
        #print(df_src.shape)
        return df_src.compare(df_trg)

        # creatin comparing dictionny
    check_lncel={}
    for i in class_lncel:
        try:
            check_lncel["check_"+i]=compare_lncel(src_cell['df_'+i],trg_cell['df_'+i])
        except ValueError as e:
            #print("Object-"+i+"- doesnt have the same size in src and trg")
            pass
     #customizing result
    liste_parent=[]
    
    for j in check_lncel.keys():
        for i in check_lncel[j].index:
            try:
                df=check_lncel[j][check_lncel[j].index==i]
                df_x=df.transpose().reset_index().pivot(index="level_0", columns="level_1", values=i)
                df_x["bande"]=i
                df_x["class"]=j.split(":")[1]
                liste_parent.append(df_x)
            except:
                print("issue when diplaying comparaison for object"+j)
    try:            
        df_cell=pd.concat(liste_parent).reset_index()

        #rename columns-cell
        col=["Parameters",trg_bts,src_bts]
        for i, j in zip(df_cell.columns[:3],col):
            df_cell.rename(columns={i:j},inplace=True)
            #print(i,j)
    except:
        print("nothing to compare")    
    
    df_bts=df_bts[df_bts["Parameters"]!="distName"]
    df_cell=df_cell[df_cell["Parameters"]!="distName"]


    #savin in excel file
    with pd.ExcelWriter('D:\scf_final_v4_'+src_bts+'_'+trg_bts+'.xlsx') as writer:  
        df_bts[["class","bande","Parameters",src_bts,trg_bts]].to_excel(writer, sheet_name='BTS-level')
        df_cell[(df_cell[src_bts].notnull()) | (df_cell[trg_bts].notnull())][["class","bande","Parameters",src_bts,trg_bts]].to_excel(writer, sheet_name="Cell-level")
        pd.concat([df_bts[["class","bande","Parameters",src_bts,trg_bts]],df_cell[(df_cell[src_bts].notnull()) | (df_cell[trg_bts].notnull())][["class","bande","Parameters",src_bts,trg_bts]]]).to_excel(writer,sheet_name="all")
        pd.DataFrame.from_dict(dictio,orient='index').transpose().to_excel(writer, sheet_name="Missing_object")
        #src_parse[src_parse.columns][1:1048576].to_excel(writer, sheet_name=src_bts+'-Parsing',index=False)
        #trg_parse[trg_parse.columns][1:1048576].to_excel(writer, sheet_name=trg_bts+'_Parsing',index=False)
    root.destroy() 
    return None

root.mainloop()

In [10]:
def Load_data():

 
    #list class reference
    class_ref=['LNBTS','ANR','ANRPRL','ANRPRW','CADPR','CAGENB','CTRLTS','DPCLPC','LBPUCCHRDPR','LNBTS_FDD','MODPR','NRDCDPR','NRDCPR','NRMEASDPR','NRMEASPR','PMRNL','RIM','SCTP','TRSNW','ULCOMP']
    class_ref=["NOKLTE:"+w for w in class_ref]
    #classes xml_file
    src_class=[elm.attrib['class'] for elm in root_src.findall(".//{raml21.xsd}managedObject")]
    trg_class=[elm.attrib['class'] for elm in root_trg.findall(".//{raml21.xsd}managedObject")]
    #intersection file
    liste_class=[x for x in class_ref if ((x in src_class) and (x in trg_class))]
    missing_class=[x for x in class_ref if x not in list(src_class)]
    missing_class
    #list class-cell-level
    class_lncel_ref=['NOKLTE:LNCEL','NOKLTE:LNCEL_FDD','NOKLTE:DRX','NOKLTE:CAREL','NOKLTE:SDRX','NOKLTE:SIB','NOKLTE:IRFIM','NOKLTE:REDRT','NOKLTE:LNHOW'] 
    class_lncel=[x for x in class_lncel_ref if ((x in src_class) and (x in trg_class))]
    missing_class_cell=[x for x in class_lncel_ref if x not in list(src_class)]





    dictio={"missing_class_bts_src":[x for x in class_ref if x not in list(src_class)],'missing_class_bts_trg':[x for x in class_ref if x not in list(trg_class)],
       'missing_class_cell_src':[x for x in class_lncel_ref if x not in list(src_class)],
       'missing_class_cell_trg':[x for x in class_lncel_ref if x not in list(trg_class)]
      }


     #check versions:
    for i,j in zip(root_src.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']"),root_trg.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']")):
        if i.attrib["version"]== j.attrib["version"]:
            msg="you're comparing same versions"
        else:
            msg="you're comparing differents versions!"
            
    #LBTS_name
    for i,j in zip(root_src.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']/{raml21.xsd}p/[@name='btsName']"),root_trg.findall(".//{raml21.xsd}managedObject/[@class='com.nokia.srbts:MRBTS']/{raml21.xsd}p/[@name='btsName']")):
        src_bts=i.text
        trg_bts=j.text
        if src_bts==trg_bts:
            trg_bts+="_V2"
             #print(src_bts)
    def file_xml(root,classe):
        for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list"):
                #print(i.attrib)
                for el in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list/[@name='"+j.attrib["name"]+"']/{raml21.xsd}item/{raml21.xsd}p"):
                    el.attrib["name"]=j.attrib['name']+"-"+el.attrib["name"]
                    #print(el.attrib["name"])    
        dicte=defaultdict(list)
   
        df_lnbts=pd.DataFrame()
        for elm in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            dicte['distName'].append(elm.attrib['distName'])
            dicte['class'].append(elm.attrib['class'])
            for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/{raml21.xsd}p"):
                dicte[i.attrib["name"]].append(i.text)
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/{raml21.xsd}list/{raml21.xsd}item/{raml21.xsd}p"):
                dicte[j.attrib["name"]].append(j.text)
            
        df=pd.DataFrame.from_dict(dicte,orient='index').transpose().fillna(method='ffill').reset_index()
        class_bts={"NOKLTE:LNBTS":"reqFreqBands-bandNumber","NOKLTE:NRDCPR":"dynTriggerLteNrDcConfList-lcrId","NOKLTE:NRDCDPR":"dynTriggerLteNrDcConfList-lcrId"} 
        
        if (classe in list(class_bts.keys())):
            if (class_bts[classe] in df.columns):
                #print(df[class_bts[classe]])
                df['bande']=">"+df[class_bts[classe]] 
            else:
                df['bande']=classe+"-"+df['index'].astype('str')
            
        else:
            df['bande']=df["distName"].map(lambda x: x.split('/')[-1])+"-"+df['index'].astype('str')
        return df.set_index("bande")

      #dictionnary of dfs for source

    src={}
    for i in liste_class:
        src["df_"+i]=file_xml(root_src,i)
    #print(src["df_NOKLTE:LNBTS"])
    
     #dictionnary of dfs for target 
    trg={}
    for i in liste_class:
        trg["df_"+i]=file_xml(root_trg,i)
    
    def _compare_(df_src,df_trg):
        
        
        
        class_bts={"NOKLTE:LNBTS":"reqFreqBands-bandNumber","NOKLTE:NRDCPR":"dynTriggerLteNrDcConfList-lcrId","NOKLTE:NRDCDPR":"dynTriggerLteNrDcConfList-lcrId"} 
        classe=df_src["class"].unique()[0]
        df_src=df_src.reset_index()
        df_trg=df_trg.reset_index()
        print(classe)
        if (classe in list(class_bts.keys())):
            print(class_bts[classe])
            if (class_bts[classe] in df_src.columns) and (class_bts[classe] in df_trg.columns):
                 
                #print(df[class_bts[classe]])
                
                df_src['bande']=">"+df_src[class_bts[classe]] 
                df_trg['bande']=">"+df_trg[class_bts[classe]] 
                print(df_trg.head())
            else:
                df_src['bande']=classe+"-"+df_src['index'].astype('str')
                df_trg['bande']=classe+"-"+df_trg['index'].astype('str')
                print(df_trg.head())
            df_src=df_src.set_index('bande')
            df_trg=df_trg.set_index('bande')
        #print(df_src.set_index.head(1))    
        #print(df_trg.set_index.head(1))
        
        df_trg.drop(df_trg[df_trg.index.isin(list(set(df_src.index).symmetric_difference(df_trg.index)))].index,inplace=True) 
        df_src.drop(df_src[df_src.index.isin(list(set(df_trg.index).symmetric_difference(df_src.index)))].index,inplace=True) 

    
        for i in list(df_trg.columns.difference(df_src.columns)):
            if df_trg.columns.get_loc(i)>len(df_src.columns):
                df_src[i]=None
            else :
                df_src.insert(df_trg.columns.get_loc(i),i,None)
             
        #print(df_src)
        for i in list(df_src.columns.difference(df_trg.columns)):
            #print(df_src.columns.get_loc(i))
            if df_src.columns.get_loc(i)>len(df_trg.columns):
                df_trg[i]=None
            else :
                df_trg.insert(df_src.columns.get_loc(i),i,None)
    
        df_trg=df_trg[df_src.columns]
        df_trg=df_trg.reindex(df_src.index)
            
        """df_src=df_src[df_src["distName"].notnull()]
        df_trg=df_trg[df_trg["distName"].notnull()]"""
    
        df_trg=df_trg[df_src.columns]
        return df_src.compare(df_trg)

    check={}
    for i in liste_class:
        try:
            check["check_"+i]=_compare_(src['df_'+i],trg['df_'+i])
        except ValueError as e:
            #print("Object-"+i+"- doesnt have the same size in src and trg")
            pass
    #print(check.keys())

    # delete columns with  just name differences

    keys=list(check.keys())
    for i in keys:
        #print(i)
        if len(check[i].columns)<=2:
            del check[i]
    

     #custolmizing result
    liste_bts=[]
    for j in check.keys():
        for i in check[j].index:
            try:
                df=check[j][check[j].index==i]
                df_y=df.transpose().reset_index().pivot(index="level_0", columns="level_1", values=i)
                df_y["bande"]=i
                df_y["class"]=j.split(":")[1]
                liste_bts.append(df_y)
            except:
                #print("Small issue in displaying result for object:"+j)
                pass
    #print(pd.concat(liste_bts).reset_index())
        
    try:        
        df_bts=pd.concat(liste_bts).reset_index()
        
        #rename columns
        col=["Parameters",trg_bts,src_bts]
        for i, j in zip(df_bts.columns[:-1],col):
            df_bts.rename(columns={i:j},inplace=True)
            #print(df_bts.columns[:-1])
        df_bts = df_bts.drop_duplicates(subset=['Parameters',trg_bts,src_bts,'class'], keep="first")
    except:
        print("result of comparaison is empty or you're comparing the same files")
    #lncell part

    def file_xml_lncel(root,classe):
        for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list"):
                for el in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+i.attrib['distName']+"']/{raml21.xsd}list/[@name='"+j.attrib["name"]+"']/{raml21.xsd}item/{raml21.xsd}p"):
                    el.attrib["name"]=j.attrib['name']+"-"+el.attrib["name"]
                
    
        df=pd.DataFrame()
        for elm in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']"):
            dicte=defaultdict(list)
            dicte['distName'].append(elm.attrib['distName'])
            dicte['class'].append(elm.attrib['class'])
            for i in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+elm.attrib['distName']+"']/{raml21.xsd}p"):
                dicte[i.attrib["name"]].append(i.text)
            for j in root.findall(".//{raml21.xsd}managedObject/[@class='"+classe+"']/[@distName='"+elm.attrib['distName']+"']/{raml21.xsd}list/{raml21.xsd}item/{raml21.xsd}p"):
                dicte[j.attrib["name"]].append(j.text)
            df=df.append(pd.DataFrame.from_dict(dicte,orient='index').transpose(),ignore_index=True)
        df=df[df["distName"].notnull()]
        
        class_ho={"NOKLTE:CAREL":"lcrId","NOKLTE:IRFIM":"dlCarFrqEut","NOKLTE:LNHOW":"utraCarrierFreq","NOKLTE:REDRT":"redirFreqUtra"}
    
        df['bande']=df["distName"].map(lambda x: x.split('LNCEL-')[-1])
        #print(df.head())
    
        if classe in list(class_ho.keys()):
            df['bande']=df['bande'].str[:2]+">"+df[class_ho[classe]]
        df = df.set_index('bande')[df.set_index('bande').index.notnull()]
        return df

    class_lncel=['NOKLTE:LNCEL','NOKLTE:LNCEL_FDD','NOKLTE:DRX','NOKLTE:SDRX','NOKLTE:CAREL','NOKLTE:SIB','NOKLTE:IRFIM','NOKLTE:REDRT','NOKLTE:LNHOW'] 

    #dictionnary of dfs for sources&targets
    src_cell={}
    trg_cell={}
    for i in class_lncel:
        src_cell["df_"+i]=file_xml_lncel(root_src,i)
        trg_cell["df_"+i]=file_xml_lncel(root_trg,i)
    print(src_cell['df_NOKLTE:LNCEL'])
        #print(src_cell["df_"+i]["distName"])

    #comparing lncel level

    def compare_lncel(df_src,df_trg):
        #print(list(df_src.index))
        df_trg.drop(df_trg[df_trg.index.isin(list(set(df_src.index).symmetric_difference(df_trg.index)))].index,inplace=True) 
        df_src.drop(df_src[df_src.index.isin(list(set(df_trg.index).symmetric_difference(df_src.index)))].index,inplace=True) 

        for i in list(df_trg.columns.difference(df_src.columns)):
            if df_trg.columns.get_loc(i)>len(df_src.columns):
                df_src[i]=None
            else :
                df_src.insert(df_trg.columns.get_loc(i),i,None)
             
        for i in list(df_src.columns.difference(df_trg.columns)):
            if df_src.columns.get_loc(i)>len(df_trg.columns):
                df_trg[i]=None
            else :
                df_trg.insert(df_src.columns.get_loc(i),i,None)  
        #print(df_trg["class"])    
        #print(len(df_trg.columns))
        df_trg=df_trg[df_src.columns]
        df_trg=df_trg.reindex(df_src.index)
        #print(df_trg.shape)
        #print(df_src.shape)
        return df_src.compare(df_trg)

        # creatin comparing dictionny
    check_lncel={}
    for i in class_lncel:
        try:
            check_lncel["check_"+i]=compare_lncel(src_cell['df_'+i],trg_cell['df_'+i])
        except ValueError as e:
            #print("Object-"+i+"- doesnt have the same size in src and trg")
            pass
     #custolmizing result
    liste_parent=[]
    
    for j in check_lncel.keys():
        for i in check_lncel[j].index:
            try:
                df=check_lncel[j][check_lncel[j].index==i]
                df_x=df.transpose().reset_index().pivot(index="level_0", columns="level_1", values=i)
                df_x["bande"]=i
                df_x["class"]=j.split(":")[1]
                liste_parent.append(df_x)
            except:
                print("issue when diplaying comparaison for object"+j)
    try:            
        df_cell=pd.concat(liste_parent).reset_index()

        #rename columns-cell
        col=["Parameters",trg_bts,src_bts]
        for i, j in zip(df_cell.columns[:3],col):
            df_cell.rename(columns={i:j},inplace=True)
            #print(i,j)
    except:
        print("nothing to compare")    
    
    df_bts=df_bts[df_bts["Parameters"]!="distName"]
    df_cell=df_cell[df_cell["Parameters"]!="distName"]


    #savin in excel file
    with pd.ExcelWriter('D:\scf_new '+src_bts+'_'+trg_bts+'.xlsx') as writer:  
        df_bts[["class","bande","Parameters",src_bts,trg_bts]].to_excel(writer, sheet_name='BTS-level')
        #df_cell[(df_cell[src_bts].notnull()) | (df_cell[trg_bts].notnull())][["class","bande","Parameters",src_bts,trg_bts]].to_excel(writer, sheet_name="Cell-level")
        #pd.concat([df_bts[["class","bande","Parameters",src_bts,trg_bts]],df_cell[(df_cell[src_bts].notnull()) | (df_cell[trg_bts].notnull())][["class","bande","Parameters",src_bts,trg_bts]]]).to_excel(writer,sheet_name="all")
        pd.DataFrame.from_dict(dictio,orient='index').transpose().to_excel(writer, sheet_name="Missing_object")
        print(df_bts[["class","bande","Parameters",src_bts,trg_bts]])
    return None

root.mainloop()

In [ ]:
df_bts.head()

In [ ]:
parsing files
def parsing_xml(file_bts,file_cell):
    file={**file_bts,**file_cell}
    s=[]
    liste=[]
    pars_src=pd.DataFrame()
    for j in file.keys():
        for i,row in file[j].iterrows():
            df=row.transpose()
            s.append(df)
            pars_bts=pd.concat(s).reset_index()
            
            pars_bts["bande"]=i
            pars_bts["class"]=j.split(":")[-1]
            liste.append(pars_bts)
    return(pd.concat(liste).reset_index())

src_parse=parsing_xml(src,src_cell)
trg_parse=parsing_xml(trg,trg_cell)

# rename columns
try:
    for i, j in zip(src_parse.columns[1:3],["Parameters",src_bts]):
        src_parse.rename(columns={i:j},inplace=True)
    for i, j in zip(trg_parse.columns[1:3],["Parameters",trg_bts]):
        trg_parse.rename(columns={i:j},inplace=True)
except:
    print("error, syntax in columns, or empty files")
    
src_parse[src_parse.columns[1:]].to_excel(writer, sheet_name=src_bts+'-Parsing',index=False)
    trg_parse[trg_parse.columns[1:]].to_excel(writer, sheet_name=trg_bts+'-Parsing',index=False)

    

In [ ]:
#class NRBTS 5G
class_ref=['NRBTS','NRDRB_RLC_AM','NRSCTP','TRACKINGAREA','BWP_PROFILE','PDCCH','NRDRB_PDCP','NRDRB','NRDRB_5QI',
            'PDCCH_CONFIG_COMMON','PDSCH','PDCCH_CONFIG_DEDICATED','NRDRB_QCI','NRANR','NRANRPR','NRSYSINFO_PROFILE_NSA',
            'com.nokia.srbts.tnl:TNLSVC','com.nokia.srbts.tnl:PMTNLINT','com.nokia.srbts.tnl:TNL',
            'com.nokia.srbts.tnl:ETHSVC','com.nokia.srbts.tnl:ETHIF',\
     'com.nokia.srbts.tnl:VLANIF','com.nokia.srbts.tnl:ETHLK','com.nokia.srbts.tnl:L2SWI','com.nokia.srbts.tnl:BRGPRT',\
     'com.nokia.srbts.tnl:IBRGPRT','com.nokia.srbts.tnl:IPNO','com.nokia.srbts.tnl:IPIF','com.nokia.srbts.tnl:IPADDRESSV4',\
     'com.nokia.srbts.tnl:IPRT','com.nokia.srbts.tnl:IPRTV6','com.nokia.srbts.tnl:QOS','com.nokia.srbts.tnl:DSCP2PCPMAP',\
     'com.nokia.srbts.tnl:DSCP2QMAP','com.nokia.srbts.tnl:FSTSCH','com.nokia.srbts.tnl:ETHAPP','com.nokia.srbts.tnl:OAMMD'])

In [ ]:
#class NRCell 5G  
class_lncel_ref=['NRCELL','NRCELL_FDD','NRDSSLTE','NRCELLGRP','NRDRB','NRDRB_MAC','NRDU','NRULCLPC_PROFILE','NRCUUP'] 

In [ ]:
#class LNBTS 4G
class_ref=['LNBTS','ANR','ANRPRL','ANRPRW','CADPR','CAGENB','CTRLTS','DPCLPC','LBPUCCHRDPR','LNBTS_FDD','MODPR','NRDCDPR',
           'NRDCPR','NRMEASDPR','NRMEASPR','PMRNL','RIM','SCTP','TRSNW','ULCOMP']

In [ ]:
#class LNCEL 4G
class_lncel_ref=['NOKLTE:LNCEL','NOKLTE:LNCEL_FDD','NOKLTE:DRX','NOKLTE:CAREL','NOKLTE:SDRX','NOKLTE:SIB','NOKLTE:IRFIM',
                 'NOKLTE:REDRT','NOKLTE:LNHOW']

In [ ]:
#lncel object that depends on paramater
class_ho={"NOKLTE:CAREL":"lcrId","NOKLTE:IRFIM":"dlCarFrqEut","NOKLTE:LNHOW":"utraCarrierFreq",
          "NOKLTE:REDRT":"redirFreqUtra"}

In [30]:
check

NameError: name 'check' is not defined